In [5]:
import numpy as np
import pandas as pd
import cassiopeia as cass
import datetime

from csv import writer


In [6]:
def get_stats(lane,participant,side,df):

    time_5 = "05:00"
    time_10 = "10:00"
    time_15 = "15:00"

    if side == 100:
        side_letter = 'B'
    elif side == 200:
        side_letter = 'R'

    if lane == 'top_lane':
        role = 'Top'
    elif lane == 'jungle':
        role = 'Jg'
    elif lane == 'mid_lane':
        role = 'Mid'
    elif lane == 'bot_lane':
        role = 'Bot'
    elif lane == 'utility':
        role = 'Sup'

    df[side_letter+'_'+role] = participant.champion.name
    df[side_letter+'_'+role+'_Kills'] = participant.stats.kills
    df[side_letter+'_'+role+'_KDA'] = participant.stats.kda

    p_state = participant.cumulative_timeline[time_5]
    df[side_letter+'_'+role+'_Kills_5'] = p_state.kills

    p_state = participant.cumulative_timeline[time_10]
    df[side_letter+'_'+role+'_Kills_10'] = p_state.kills

    p_state = participant.cumulative_timeline[time_15]
    df[side_letter+'_'+role+'_Kills_15'] = p_state.kills

In [7]:
def add_match_history(puuid, games):
    global df, df2
    match_history = cass.MatchHistory(puuid=puuid,continent="AMERICAS", region='NA',begin_index=0,end_index=games,queue=cass.Queue.ranked_solo_fives)
    
    for match in match_history:
        try:
            stat_import = pd.DataFrame(columns=df.columns,index=[0])
        except:
            stat_import = pd.DataFrame(index=[0])
            
        try:
            if str(match.id) in df["ID"].values or str(match.id)[4:] in df["ID"].values:
                continue
        except:
            pass
        
        if match.duration < datetime.timedelta(hours=0, minutes=15, seconds=0):
            continue
        
        player_list = []
        # Making the player objects and adding to list above
        for i in range(0,10):
            # pi = participant i
            exec('p'+str(i)+'='+'match.participants['+str(i)+']')
            # add pi to list
            exec('player_list.append(p'+str(i)+')')
        
        if type(match.id) == str:
            stat_import['ID'] = match.id[4:]
        else:
            stat_import['ID'] = str(match.id)
        stat_import['B_Win'] = match.blue_team.win

        try:
            
             
            for player in player_list:
                
                if player.side.value == 100:
                    if player.individual_position.name == 'top_lane':
                        get_stats(player.individual_position.name,player,player.side.value,stat_import)
                    elif player.individual_position.name == 'jungle':
                        get_stats(player.individual_position.name,player,player.side.value,stat_import)
                    elif player.individual_position.name == 'mid_lane':
                        get_stats(player.individual_position.name,player,player.side.value,stat_import)
                    elif player.individual_position.name == 'bot_lane':
                        get_stats(player.individual_position.name,player,player.side.value,stat_import)
                    elif player.individual_position.name == 'utility':
                        get_stats(player.individual_position.name,player,player.side.value,stat_import)
                if player.side.value == 200:
                    if player.individual_position.name == 'top_lane':
                        get_stats(player.individual_position.name,player,player.side.value,stat_import)
                    elif player.individual_position.name == 'jungle':
                        get_stats(player.individual_position.name,player,player.side.value,stat_import)
                    elif player.individual_position.name == 'mid_lane':
                        get_stats(player.individual_position.name,player,player.side.value,stat_import)
                    elif player.individual_position.name == 'bot_lane':
                        get_stats(player.individual_position.name,player,player.side.value,stat_import)
                    elif player.individual_position.name == 'utility':
                        get_stats(player.individual_position.name,player,player.side.value,stat_import)
                        
            # stat_import.to_csv(path_or_buf="Match_Data/matches2.csv",mode='a',index=False,header=False)
                    
            df = pd.concat([df,stat_import], ignore_index=True)
            df2 = pd.concat([df2,stat_import], ignore_index=True)
                
        except:
           pass

    return find_next_player(match_history)
    

In [8]:
# Function that selects a random player from the last player's previous games
def find_next_player(history):
    game = 0
    New_Player_Attempts = 0
    while True:
        New_Player_Attempts += 1
        # the match to select a new player from
        match = history[game]
        # selecting random number to get the random player from match
        randNum = np.random.randint(11)
        player = match.participants[randNum]
        # If player has already been checked
        if player.summoner.name in already_checked:
            # only 10 players in a match, if all players been checked go to next game
            if New_Player_Attempts == 10:
                game += 1
                New_Player_Attempts = 0
                continue
            else:
                continue
        break
    print(player.summoner.name)
    # keep track of the account that has been checked
    already_checked.append(player.summoner.name)
    # Returns the puuid of the new player
    return player.summoner.puuid

In [9]:
cass.set_riot_api_key("RGAPI-e22a2f61-a2fb-4553-b8b8-9216f7bff4d4")
player = cass.Summoner(name="Artezzaa", region="NA")


In [6]:
i = 1
# list so we don't check the same player's games more than once
already_checked = [player.name]
# DataFrame that keeps track of all the match data over whole process
df = pd.DataFrame()
# Get the match info for the seed player, with this number of matches
next_puuid = add_match_history(player.puuid, 20)
# DataFrame to output blocks of matches at once
df2 = pd.DataFrame(df)
# Print out the first block of matches from seed account with column names
df2.to_csv("Match_Data/matches.csv",index=False,mode='w')
# Number of players you want to get matches from
while i != 10:
    # empty output dataframe to stop duplicates in file
    df2 = df2.iloc[0:0]
    # Get the match info for the randomly selected player, with this number of matches
    next_puuid = add_match_history(next_puuid, 20)
    # Print out the next block of matches from random selected account without columns
    df2.to_csv("Match_Data/matches.csv",index=False,mode='a',header=False)
    i += 1

Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/Artezzaa
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/TbAmHuYgAZxeaiwLN-ScDFTKuksDimRmnSgLe4EsW_3CS-0HUCIE5LqTwZk-Mp2lhQZ7SPsDMUIgXw/ids?start=0&count=20&queue=420
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4254552847
Making call: https://cdn.merakianalytics.com/riot/lol/resources/patches.json
Making call: https://ddragon.leagueoflegends.com/api/versions.json
Making call: https://ddragon.leagueoflegends.com/cdn/12.5.1/data/en_US/championFull.json
Making call: https://ddragon.leagueoflegends.com/realms/na.json
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4254552847/timeline
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4251572080
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4251572080/timeline
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_42

In [ ]:
# https://github.com/meraki-analytics/cassiopeia/tree/master/examples